# Project 4 - Behavioral Cloning
### Lukasz Uszko, June 2019

## Summary
This report describes my implementation of a convolutional neural network (CNN) to predict steering angles from a set of training images acquired from a driving simulator. The CNN is a slightly modified Keras implementation of this paper from NVIDIA Corporation. The trained model successfully steered the simulated car around track-1 of the beta-simulator provided as part of this project.

This report describes results from training performed using the sample dataset provided by Udacity and my own training set from tracks 1 and 2 which I acquired by manually driving the car.

## Submission
The following files accompany this report:

* `model.py`: Script used to create and train the CNN. This report describes how the model was developed and trained.
* `model.h5`: The trained model, created by the above script.
* `drive.py`: Script used to drive the car using the trained model above.
## Usage
* Open a terminal and start the simulator: ./beta_simulator.x86_64
* Open another terminal and run the following to drive the simulated car autonomously: python drive.py model.h5
* The simulated car will now start moving. Verify that the car stays on the road at all times

## Imports and project globals

In [13]:
# Imports
import pickle
import math
import glob
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
import skimage
import csv

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from random import shuffle
from tqdm import tqdm

# Visualizations will be shown in the notebook.
%matplotlib inline

# Some globals
DATA_FOLDER_PATH = './data'
A4_PORTRAIT = (8.27, 11.69)
A4_LANDSCAPE = A4_PORTRAIT[::-1]

## 1. Model Development
### Loading the training set
I first examined the organisation of training data which consisted of a csv file and a set of images. The following code loads and prints out some basic information about training set.

In [14]:
# Read driving_log data
csv_file_path = os.path.join(DATA_FOLDER_PATH,'driving_log.csv')
samples = []
with open(csv_file_path) as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        samples.append(line)

csvfile = pd.read_csv(csv_file_path)
print(csvfile.center.values)
print(csvfile.left.values)
print(csvfile.right.values)


# Split data
train_samples, validation_samples = train_test_split(samples, test_size=0.2)

def load_image(filename):
    image = cv2.imread(filename)
    return cv2.cvtColor(image,cv2.COLOR_BGR2RGB)

['IMG/center_2016_12_01_13_30_48_287.jpg'
 'IMG/center_2016_12_01_13_30_48_404.jpg'
 'IMG/center_2016_12_01_13_31_12_937.jpg' ...,
 'IMG/center_2019_06_28_17_16_51_129.jpg'
 'IMG/center_2019_06_28_17_16_51_211.jpg'
 'IMG/center_2019_06_28_17_16_51_333.jpg']
[' IMG/left_2016_12_01_13_30_48_287.jpg'
 ' IMG/left_2016_12_01_13_30_48_404.jpg'
 ' IMG/left_2016_12_01_13_31_12_937.jpg' ...,
 'IMG/left_2019_06_28_17_16_51_129.jpg'
 'IMG/left_2019_06_28_17_16_51_211.jpg'
 'IMG/left_2019_06_28_17_16_51_333.jpg']
[' IMG/right_2016_12_01_13_30_48_287.jpg'
 ' IMG/right_2016_12_01_13_30_48_404.jpg'
 ' IMG/right_2016_12_01_13_31_12_937.jpg' ...,
 'IMG/right_2019_06_28_17_16_51_129.jpg'
 'IMG/right_2019_06_28_17_16_51_211.jpg'
 'IMG/right_2019_06_28_17_16_51_333.jpg']


### Training set augmentation

During training, you want to feed the left and right camera images to your model as if they were coming from the center camera. This way, you can teach your model how to steer if the car drifts off to the left or the right.

Figuring out how much to add or subtract from the center angle will involve some experimentation.

During prediction (i.e. "autonomous mode"), you only need to predict with the center camera image.

It is not necessary to use the left and right images to derive a successful model. Recording recovery driving from the sides of the road is also effective.